In [12]:
import numpy as np
from scipy import sparse, stats
import pandas as pd
import bottleneck
import gc
import warnings
import scanpy as sc
import anndata as ad
import time
import os
from scipy import stats, sparse
import concurrent.futures
import statistics
import random
import threading
import concurrent.futures
from scipy.sparse import csr_matrix
from scipy.stats import zscore

from pyprojroot import here
import pymn
import resource

In [13]:
here()
#base_data_folder = "/data/bican" #for the Gillis lab
#base_data_folder = "/grid/gillis/data/lfrench" #for the Elzar cluster
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [14]:
#common universe of markers
cluster_annotations = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep="\t")
all_markers = set(cluster_annotations['cluster.markers'].str.split(',').explode().tolist())
all_Zeng_markers = [x for x in all_markers if pd.notna(x)]
all_Zeng_markers = set(all_Zeng_markers)



In [15]:
#load Macosko marker lists from preprint supplement table
Macosko_supplement_table = pd.read_excel(base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.xlsx", sheet_name = "Whole Brain Set Cover")
#just the most specific markers
Macosko_supplement_table = Macosko_supplement_table[Macosko_supplement_table["Exclude up to N Nearest Neighbors"] == 0]

Macosko_supplement_table = Macosko_supplement_table[["Cell Type Name", "Optimally-Sized Gene List"]]
Macosko_markers = Macosko_supplement_table['Optimally-Sized Gene List'].str.split('|', expand=True)
Macosko_markers = pd.concat([Macosko_supplement_table['Cell Type Name'], Macosko_markers], axis=1)
Macosko_markers = pd.melt(Macosko_markers, id_vars = 'Cell Type Name', value_name='gene_symbol', var_name='order')
Macosko_markers['ensembl'] = Macosko_markers['gene_symbol'].str.replace(".*=", "", regex=True)
Macosko_markers['gene_symbol'] = Macosko_markers['gene_symbol'].str.replace("=.*", "", regex=True)
Macosko_markers.dropna(subset=['gene_symbol'], inplace=True)
Macosko_markers = Macosko_markers.rename(columns = {"gene_symbol":"gene"})
Macosko_markers = Macosko_markers.rename(columns = {"Cell Type Name":"Macosko_cluster"})


In [16]:
Macosko_markers

,Macosko_cluster,order,gene,ensembl
0,Ex_Gbx2_Adam18_2,0,Sp9,ENSMUSG00000068859
1,Inh_Pax7_Hpse_2,0,Pde11a,ENSMUSG00000075270
3,Ex_Pitx2_Zbtb7c_3,0,Kcnq5,ENSMUSG00000028033
4,Inh_Gata3_Gxylt2_5,0,Ltk,ENSMUSG00000027297
5,Inh_Lhx6_Npy_3,0,Syt6,ENSMUSG00000027849
...,...,...,...,...
36867,Ex_Ebf2_Prox1_Pou4f3,7,Trhr,ENSMUSG00000038760
37890,Inh_Lhx1_Otx2_1,7,Ankfn1,ENSMUSG00000047773
38645,Ex_Pou4f1_Plscr5_1,7,Dpf3,ENSMUSG00000021221
42880,Inh_Lhx1_Otx2_1,8,Dcdc2a,ENSMUSG00000035910


In [17]:
Macosko_markers[Macosko_markers.Macosko_cluster == "Ex_Gbx2_Adam18_2"]

,Macosko_cluster,order,gene,ensembl
0,Ex_Gbx2_Adam18_2,0,Sp9,ENSMUSG00000068859
4990,Ex_Gbx2_Adam18_2,1,Gm45194,ENSMUSG00000109156
9980,Ex_Gbx2_Adam18_2,2,Calca,ENSMUSG00000030669
14970,Ex_Gbx2_Adam18_2,3,Fbln1,ENSMUSG00000006369


In [18]:
all_Macosko_markers = set(Macosko_markers.gene)

In [19]:
len(all_Macosko_markers)

2706

In [20]:
len(all_Zeng_markers)

2725

In [23]:
base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/"

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/processed/macosko/subsets/'

In [24]:
#for when it's already been subsetted
all_Mac_cpm = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed_3309_markers_only.cpm.h5ad")

In [25]:
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad", backed = "r")
gene_map = adata_macosko.var
Macosko_universe = set(gene_map['gene_name'].to_list())

In [26]:
#join with universes
#all_Macosko_markers = all_Macosko_markers.intersection(Zeng_universe)
all_Zeng_markers = all_Zeng_markers.intersection(Macosko_universe)

In [27]:
len(all_Macosko_markers)

2706

In [28]:
len(all_Zeng_markers)

2214

In [29]:
len(all_Macosko_markers.intersection(all_Zeng_markers))

1578

In [31]:
combined_markers = all_Zeng_markers.union(all_Macosko_markers)

In [32]:
len(combined_markers)

3342

In [20]:
##subset to marker genes, and write out, used once
#all_Zeng_cpm.var["is_Marker"] = all_Zeng_cpm.var.gene_symbol.isin(combined_markers)
#all_Zeng_cpm = all_Zeng_cpm[:, all_Zeng_cpm.var["is_Marker"]]
#all_Zeng_cpm.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome_3309_markers_only.cpm.h5ad") 

In [33]:
if not os.path.isdir(os.path.join(here(), "results", "marker_results")):
  os.mkdir(os.path.join(here(), "results", "marker_results"))

In [34]:
Zeng_cluster_info_AWS = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep = '\t')
#remove low quality cells
LQ_clusters = Zeng_cluster_info_AWS.cl[Zeng_cluster_info_AWS['subclass_label'] == "LQ"].to_list()
LQ_clusters = [str(i) for i in LQ_clusters]
#LQ_clusters not used

In [35]:
all_Mac_cpm

AnnData object with n_obs × n_vars = 4407296 × 3310
    obs: 'ClusterKey', 'ClusterNm', 'derived_cell_libs'
    var: 'gene_name', 'is_marker'

In [36]:
#%%time
all_Mac_cpm_X = sparse.csr_matrix(all_Mac_cpm.X)
df = pd.DataFrame.sparse.from_spmatrix(all_Mac_cpm_X)
df.index = all_Mac_cpm.obs.index  # Assuming row labels are stored in `.obs_names`
df.columns = all_Mac_cpm.var.index

In [37]:
df.shape

(4407296, 3310)

In [38]:
#z-score normalize the cpm matrix
#%%time
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df[col] = zscore(df[col].to_numpy())

In [39]:
all_clusters_Zeng = Zeng_cluster_info_AWS.cl.tolist()
all_clusters_Zeng = [str(i) for i in all_clusters_Zeng]

In [40]:
all_clusters_Macosko = list(set(Macosko_markers.Macosko_cluster.tolist()))

In [41]:
#for testing
#all_clusters_Macosko = all_clusters_Macosko[1:100]
#ll_clusters_Zeng = all_clusters_Zeng[1:100]

In [42]:
all_clusters= all_clusters_Macosko + all_clusters_Zeng

In [43]:
len(all_clusters)

198

In [44]:
#make dict for cluster markers and cells in cluster
cluster_marker_dict = dict()
for cluster in all_clusters_Zeng:
    #get markers 
    cluster_markers = Zeng_cluster_info_AWS[Zeng_cluster_info_AWS.cl == int(cluster)][['cluster.markers']].iloc[0,0]
    if pd.isna(cluster_markers):
        continue
    cluster_markers = cluster_markers.split(',')
    #use Macosko to convert to ensmbl ID's to handle the Nrg1 mapping
    cluster_markers = Macosko_markers[Macosko_markers.gene.isin(cluster_markers)].ensembl.tolist()
    cluster_markers = set(cluster_markers).intersection(all_Mac_cpm.var.index.to_list())
    cluster_marker_dict[cluster] = cluster_markers


In [45]:
#Add in Macosko markers too
for cluster in all_clusters_Macosko:
    #get markers 
    cluster_markers = Macosko_markers[Macosko_markers.Macosko_cluster == cluster].ensembl.tolist()
    cluster_markers = set(cluster_markers).intersection(all_Mac_cpm.var.index.to_list())
    cluster_marker_dict[cluster] = cluster_markers

In [46]:
len(cluster_marker_dict)

198

In [47]:
# Flatten the dictionary and convert it to a DataFrame
cluster_marker_dict_long = pd.DataFrame([(k, v) for k, vs in cluster_marker_dict.items() for v in vs], columns=['cluster', 'gene_symbol'])

# Create a new DataFrame with 1s where the values match
cluster_marker_dict_table = cluster_marker_dict_long.pivot_table(index='cluster', columns='gene_symbol', aggfunc=len, fill_value=0)



In [48]:
#line up the df dataframe rows
cluster_sums = df[cluster_marker_dict_table.columns] @ cluster_marker_dict_table.T

In [49]:
cluster_sums

cluster,100,101,102,103,104,105,106,107,108,109,...,Inh_Six3_Cd40,Inh_Six3_Six6_1,Inh_Six3_Sox2_3,Inh_Six3_Tpbgl,Inh_Six6_Vipr2_5,Inh_Sncg_Np2yr_Fos_2,Inh_Tfap2b_Agtr1a_2,Inh_Tfap2b_Erfl_1,Inh_Usp43_Pvalb,NorEx_Phox2b_Pmnt_2
pBICCNsMMrBSL1aiM007d190529_CAGTTCCTCTATCGGA,-0.109338,-0.645102,-0.103083,-0.508128,-0.670987,0.431496,-0.153577,-0.454505,-0.376532,2.515560,...,-0.165828,0.239458,1.814146,-0.398423,-0.599612,-0.700673,0.458372,-0.439213,-0.575289,-0.510997
pBICCNsMMrBSL1aiM007d190529_GATCAGTCACCCTGTT,-0.308501,-0.149321,-0.661794,-0.508128,-0.670987,-0.350833,-0.612829,-0.454505,-0.969936,2.550968,...,-0.393490,0.920490,-0.580846,-0.398423,-0.599612,-1.041494,-0.284288,-0.534111,-0.575289,-0.510997
pBICCNsMMrBSL1aiM007d190529_CTGGCAGAGGGAACAA,-0.308501,-0.601671,-0.661794,-0.508128,-0.670987,-0.350833,1.412461,-0.454505,-0.996616,3.743462,...,-0.393490,-0.293603,-0.400653,-0.281975,-0.476958,-0.957832,3.179704,1.465541,-0.575289,-0.422274
pBICCNsMMrBSL1aiM007d190529_GTAATGCAGCGCATCC,-0.308501,-0.548478,-0.661794,0.059206,-0.670987,1.482909,-0.928516,0.486223,1.774772,0.755903,...,-0.393490,6.578477,-0.752800,-0.398423,-0.599612,-0.866894,6.678755,-0.534111,-0.575289,-0.221189
pBICCNsMMrBSL1aiM007d190529_ACTTCCGGTGGTCCCA,0.962487,0.827087,2.776826,-0.314719,-0.477577,-0.350833,-0.502255,-0.454505,-0.995884,1.356914,...,-0.393490,0.095836,0.034325,-0.398423,-0.473751,-0.792366,-0.284288,-0.534111,-0.575289,0.558749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pBICCNsMMrHPFCA2CA3iF014d210217A2_CCCTGATAGCGAGTAC,-0.308501,-0.749517,-0.661794,-0.508128,-0.670987,-0.350833,-0.942111,-0.454505,-1.024605,-0.372242,...,-0.393490,-0.293603,-0.752800,-0.398423,-0.599612,-1.407235,-0.284288,-0.534111,-0.575289,11.387682
pBICCNsMMrHPFCA2CA3iF014d210217A2_AGCGTCGTCAGACTGT,-0.308501,-0.749517,-0.661794,-0.508128,-0.670987,-0.350833,-0.942111,-0.454505,-1.024605,-0.372242,...,-0.393490,-0.293603,-0.752800,-0.398423,-0.599612,-1.407235,-0.284288,-0.534111,-0.575289,-0.510997
pBICCNsMMrHPFCA2CA3iF014d210217A2_AGCGTCGGTGAGGATC,-0.308501,-0.749517,-0.661794,-0.508128,-0.670987,-0.350833,5.926823,-0.454505,-1.024605,-0.372242,...,-0.393490,-0.293603,-0.752800,-0.398423,-0.599612,-1.407235,-0.284288,-0.534111,-0.575289,-0.510997
pBICCNsMMrHPFCA2CA3iF014d210217A2_TGCAGGCCACAAGCTT,-0.308501,-0.749517,-0.661794,-0.508128,-0.670987,-0.350833,-0.942111,-0.454505,-1.024605,-0.372242,...,-0.393490,-0.293603,7.041873,-0.398423,-0.599612,-1.407235,-0.284288,5.175826,-0.575289,-0.510997


In [50]:
#%%time
one_hot_cluster_membership = pd.get_dummies(all_Mac_cpm.obs.ClusterNm)

In [51]:
#restrict to cluster sum clusters

In [52]:
one_hot_cluster_membership = one_hot_cluster_membership[all_clusters_Macosko]

In [53]:
#AUROC code based on https://github.com/gillislab/pyMN/blob/master/pymn/utils.py

In [54]:
#%%time
ranks = bottleneck.rankdata(cluster_sums, axis = 0)

In [55]:
sum_pos_ranks = one_hot_cluster_membership.T @ ranks

In [56]:
n_pos = bottleneck.nansum(one_hot_cluster_membership, axis=0)

In [57]:
n_neg = one_hot_cluster_membership.shape[0] - n_pos

In [58]:
roc = sum_pos_ranks / n_pos[:, None]
roc -= (n_pos[:, None] + 1) / 2
roc /= n_neg[:, None]

In [59]:
roc

,0,1,2,3,4,5,6,7,8,9,...,187,188,189,190,191,192,193,194,195,196
Ex_Pitx1_Nts,0.642398,0.761285,0.569086,0.461042,0.421252,0.596406,0.617430,0.438855,0.670879,0.492349,...,0.509868,0.753784,0.662244,0.510298,0.614552,0.355990,0.529399,0.360858,0.397808,0.380386
Inh_Dmbx1_Gpx6,0.468262,0.688538,0.913296,0.447479,0.472242,0.535369,0.833831,0.710342,0.306858,0.467260,...,0.468168,0.554601,0.603052,0.451021,0.653643,0.357632,0.493106,0.762239,0.447544,0.356460
Ex_Skor1_Samd3_4,0.570357,0.803950,0.406120,0.597563,0.495458,0.464094,0.564150,0.444200,0.439431,0.462515,...,0.614266,0.616533,0.483165,0.458684,0.388715,0.523207,0.477668,0.392957,0.408884,0.621797
Inh_Pax8_Sp9_2,0.480598,0.635177,0.380137,0.511054,0.480481,0.528052,0.523915,0.454460,0.788907,0.709502,...,0.461272,0.472172,0.558377,0.527121,0.595909,0.508080,0.719787,0.639531,0.407084,0.334168
Ex_Lhx9_Igfbp2_1,0.469180,0.536799,0.883184,0.466644,0.444489,0.454534,0.479036,0.448890,0.512355,0.641057,...,0.461005,0.780946,0.474014,0.455185,0.400266,0.531511,0.703319,0.603284,0.415273,0.328880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_Lmx1a_Pgr15l_3,0.451817,0.727615,0.583077,0.563173,0.582002,0.502062,0.517808,0.438855,0.610114,0.578530,...,0.545475,0.466099,0.708930,0.532229,0.648800,0.623152,0.475494,0.391522,0.695042,0.372795
Ex_Lhx3_A730046J19Rik,0.463497,0.507250,0.393669,0.442104,0.543270,0.471923,0.639776,0.451643,0.640593,0.455956,...,0.453333,0.505428,0.467514,0.476938,0.604316,0.590392,0.481710,0.378088,0.421421,0.539254
Ex_Pitx2_Nkx2-2_2,0.494186,0.799010,0.419967,0.461605,0.479858,0.451561,0.890146,0.443389,0.390199,0.538954,...,0.463790,0.485393,0.509525,0.486509,0.517155,0.537994,0.475494,0.368566,0.459289,0.371151
Inh_Hoxb5_Tecrl,0.545021,0.634319,0.415323,0.594427,0.470570,0.659688,0.675435,0.601805,0.455165,0.689327,...,0.461352,0.477303,0.401345,0.470671,0.400586,0.517720,0.763196,0.402790,0.437157,0.397564


In [60]:
#set row and column labels

In [61]:
roc.index = roc.index.map(lambda x: x + '_cluster')
roc.columns = cluster_sums.columns.tolist()
roc.columns = roc.columns.map(lambda x: x + '_markers')

In [62]:
roc

,100_markers,101_markers,102_markers,103_markers,104_markers,105_markers,106_markers,107_markers,108_markers,109_markers,...,Inh_Six3_Cd40_markers,Inh_Six3_Six6_1_markers,Inh_Six3_Sox2_3_markers,Inh_Six3_Tpbgl_markers,Inh_Six6_Vipr2_5_markers,Inh_Sncg_Np2yr_Fos_2_markers,Inh_Tfap2b_Agtr1a_2_markers,Inh_Tfap2b_Erfl_1_markers,Inh_Usp43_Pvalb_markers,NorEx_Phox2b_Pmnt_2_markers
Ex_Pitx1_Nts_cluster,0.642398,0.761285,0.569086,0.461042,0.421252,0.596406,0.617430,0.438855,0.670879,0.492349,...,0.509868,0.753784,0.662244,0.510298,0.614552,0.355990,0.529399,0.360858,0.397808,0.380386
Inh_Dmbx1_Gpx6_cluster,0.468262,0.688538,0.913296,0.447479,0.472242,0.535369,0.833831,0.710342,0.306858,0.467260,...,0.468168,0.554601,0.603052,0.451021,0.653643,0.357632,0.493106,0.762239,0.447544,0.356460
Ex_Skor1_Samd3_4_cluster,0.570357,0.803950,0.406120,0.597563,0.495458,0.464094,0.564150,0.444200,0.439431,0.462515,...,0.614266,0.616533,0.483165,0.458684,0.388715,0.523207,0.477668,0.392957,0.408884,0.621797
Inh_Pax8_Sp9_2_cluster,0.480598,0.635177,0.380137,0.511054,0.480481,0.528052,0.523915,0.454460,0.788907,0.709502,...,0.461272,0.472172,0.558377,0.527121,0.595909,0.508080,0.719787,0.639531,0.407084,0.334168
Ex_Lhx9_Igfbp2_1_cluster,0.469180,0.536799,0.883184,0.466644,0.444489,0.454534,0.479036,0.448890,0.512355,0.641057,...,0.461005,0.780946,0.474014,0.455185,0.400266,0.531511,0.703319,0.603284,0.415273,0.328880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_Lmx1a_Pgr15l_3_cluster,0.451817,0.727615,0.583077,0.563173,0.582002,0.502062,0.517808,0.438855,0.610114,0.578530,...,0.545475,0.466099,0.708930,0.532229,0.648800,0.623152,0.475494,0.391522,0.695042,0.372795
Ex_Lhx3_A730046J19Rik_cluster,0.463497,0.507250,0.393669,0.442104,0.543270,0.471923,0.639776,0.451643,0.640593,0.455956,...,0.453333,0.505428,0.467514,0.476938,0.604316,0.590392,0.481710,0.378088,0.421421,0.539254
Ex_Pitx2_Nkx2-2_2_cluster,0.494186,0.799010,0.419967,0.461605,0.479858,0.451561,0.890146,0.443389,0.390199,0.538954,...,0.463790,0.485393,0.509525,0.486509,0.517155,0.537994,0.475494,0.368566,0.459289,0.371151
Inh_Hoxb5_Tecrl_cluster,0.545021,0.634319,0.415323,0.594427,0.470570,0.659688,0.675435,0.601805,0.455165,0.689327,...,0.461352,0.477303,0.401345,0.470671,0.400586,0.517720,0.763196,0.402790,0.437157,0.397564


In [63]:
timestamp = str(round(time.time()))
roc.rename_axis('cluster_name').to_csv(os.path.join(here(), "results", "marker_results", "markers_on_Macosko." + timestamp + ".csv.gz"), compression="gzip")

In [64]:
#run best versus self

In [65]:
#needs to iterate all marker sets, get the two clusters, then compute AUROC 

In [66]:
roc_used_marker_lists = [string.replace("_markers", "") for string in roc.columns.tolist()]

In [67]:
all_clusters_Macosko = set(all_clusters_Macosko).intersection(set(roc_used_marker_lists))

In [68]:
all_results = []
for current_marker in all_clusters_Macosko:
    roc_series = roc[current_marker + "_markers"].copy()
    self_auroc = roc_series.loc[current_marker + "_cluster"] #only used for output
    #set to zero so it doesn't mask the non-self max
    roc_series.loc[current_marker + "_cluster"] = 0
    highest_non_self_cluster = roc_series.idxmax()
    two_cluster_membership = one_hot_cluster_membership[[highest_non_self_cluster.replace("_cluster", ""), current_marker]]
    two_cluster_membership = two_cluster_membership[two_cluster_membership.sum(axis=1) > 0]
    cluster_sums_self = cluster_sums.loc[two_cluster_membership.index, current_marker]
    ranks = bottleneck.rankdata(cluster_sums_self, axis = 0)
    sum_pos_ranks = two_cluster_membership[current_marker].T @ ranks
    n_pos = bottleneck.nansum(two_cluster_membership[current_marker], axis=0)
    if n_pos == 0:
        continue
    n_neg = two_cluster_membership[current_marker].shape[0] - n_pos
    single_roc = sum_pos_ranks / n_pos
    single_roc -= (n_pos + 1) / 2
    single_roc /= n_neg
    single_cluster_result = pd.DataFrame({'target_cluster' : current_marker,
                                      'highest_non_self_cluster' : highest_non_self_cluster,
                                      'max_AUROC' : roc_series.max(),
                                      'self_1_vs_all_AUROC' : self_auroc,
                                      'self_vs_best_AUROC' : single_roc}, index=[0]) 
    all_results.append(single_cluster_result)

all_results = pd.concat(all_results)

In [69]:
all_results

,target_cluster,highest_non_self_cluster,max_AUROC,self_1_vs_all_AUROC,self_vs_best_AUROC
0,Ex_Pitx1_Nts,Ex_Ebf2_Ifi213_2_cluster,0.969841,0.907843,0.569740
0,Inh_Dmbx1_Gpx6,Inh_Frmd7_Olfr142_10_cluster,0.752370,0.846924,0.597431
0,Ex_Skor1_Samd3_4,Ex_Onecut2_Hmx3_Dmbx1_2_cluster,0.940141,0.970869,0.787079
0,Inh_Pax8_Sp9_2,Inh_Pax6_Pdgfd_5_cluster,0.972665,0.953096,0.436866
0,Ex_Lhx9_Igfbp2_1,Ex_Pou4f1_Htr5b_7_cluster,0.994272,0.959108,0.640851
...,...,...,...,...,...
0,Ex_Lmx1a_Pgr15l_3,Inh_Six3_Cd40_cluster,0.896383,0.969741,0.667861
0,Ex_Lhx3_A730046J19Rik,Inh_Pax8_Sp9_2_cluster,0.861564,0.856768,0.618817
0,Ex_Pitx2_Nkx2-2_2,Inh_Pax2_Hoxb5_Rubcnl_2_cluster,0.923414,0.932652,0.863636
0,Inh_Hoxb5_Tecrl,Inh_Onecut3_Tbpl2_cluster,0.824499,0.830035,0.519526


In [70]:
all_results.to_csv(os.path.join(here(), "results", "marker_results", "Macosko_self_vs_best." + timestamp + ".csv.gz"), compression="gzip", index=False)

In [71]:
print("Done self vs rest")

Done self vs rest
